# Communications point-à-point

* Elles sont _locales_, c'est-à-dire que seuls **deux
  processus** sont impliqués : ils sont les seuls à
  appeler réciproquement des fonctions de communication,
  donc ils sont les seuls à participer à cet échange
    * Note : les appels aux fonctions d'envoi et de réception
      viennent toujours par paires sur des processus différents
* Les fonctions d’envoi et de réception peuvent être bloquantes ou non

## `MPI_Send()` (bloquant)

En C :

```C
int MPI_Send(void *envoi, int compte, MPI_Datatype type,
             int dest, int etiquette, MPI_Comm comm);
```

* `envoi` : adresse en mémoire du premier élément à envoyer
* `compte` : nombre d’éléments de type `type`
* `type` : type MPI tel que `MPI_INT`, `MPI_DOUBLE`, etc.
* `dest` : rang du processus recevant
* `etiquette` : nombre entier identifiant le type de transfert

En Python :

```Python
comm.send(objet, dest=dest, tag=etiquette)
```

* `objet` : n'importe quel objet sérialisable via
  [`pickle`](https://docs.python.org/3/library/pickle.html#module-pickle)

## `MPI_Recv()` (bloquant)

En C :

```C
int MPI_Recv(void *recept, int compte, MPI_Datatype type,
             int source, int etiquette, MPI_Comm comm,
             MPI_Status *etat);
```

* `recept` : adresse en mémoire du premier élément à recevoir
* `compte` : nombre maximal d'éléments de type `type`
* `source` : rang du processus transmettant,
  ça peut être `MPI_ANY_SOURCE`
* `etiquette` : ça peut être `MPI_ANY_TAG`
* `etat` : informations sur le transfert
    * `.count` : nombre d’éléments reçus de type `type`
    * `.MPI_SOURCE` : rang de la source des données
    * `.MPI_TAG` : étiquette du transfert

En Python :

```Python
objet = comm.recv(source=source, tag=etiquette, status=etat)
```

* `objet` : une variable ou une partie d'un objet modifiable,
  pour recevoir l'objet désérialisé
* `source` : `MPI.ANY_SOURCE` (valeur par défaut) ou un rang précis
* `etiquette` : `MPI.ANY_TAG` (valeur par défaut) ou un nombre précis
* `etat` : `None` (valeur par défaut) ou objet de type `MPI.Status`
    * `.count` : nombre d’octets reçus
    * `.source` : rang de la source des données
    * `.tag` : étiquette du transfert

### Exemple - `MPI_Send` et `MPI_Recv`

Chaque processus a ses propres variables `a` et `b`.
Le processus 2 envoie la valeur de son `a` vers
le processus 0 qui reçoit cette valeur via son `b`.

![Figure - Communication point à point](images/mpi_point2point.svg)

En C :

```C
if (proc == 2) {
    MPI_Send(&a, 1, MPI_INTEGER, 0, 746, MPI_COMM_WORLD);
}
else if (proc == 0) {
    MPI_Recv(&b, 1, MPI_INTEGER, 2, 746, MPI_COMM_WORLD, &etat);
}
```

En Python :

```Python
if proc == 2:
    MPI.COMM_WORLD.send(a, dest=0, tag=746)
elif proc == 0:
    b = MPI.COMM_WORLD.recv(source=2, tag=746)
```

### Exercice #2 - Envoi d'une matrice

Votre objectif : envoyer une matrice 4x4 du processus 0 au processus 1 :

1. Dans le dossier `exercices`, éditez le fichier `send_matrix.c`
   (ou `.py`) pour programmer le transfert de la matrice
1. Compilez le code et lancez-le avec deux (2) processus